In [31]:
import pandas as pd
import numpy as np
import pickle
import joblib

from tkinter import filedialog
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn import metrics
from sklearn.metrics import classification_report

In [2]:
file1 = filedialog.askopenfilename()
rawdata = pd.read_csv(file1, sep=",",  skip_blank_lines=True)#inputfile
rawdata

,Unnamed: 0,Acceleration_in_X,Acceleration_in_Y,Acceleration_in_Z,Roll,Pitch,Yaw,Lane_change,turning,braking,acceleration
0,0,-0.005,0.008,0.018,-1.523,0.015,0.012,0.0,0.0,0.0,0.0
1,1,0.016,-0.002,0.018,-1.522,0.012,0.012,0.0,0.0,0.0,0.0
2,2,0.037,-0.005,0.018,-1.520,0.014,0.011,0.0,0.0,0.0,0.0
3,3,0.038,-0.009,0.024,-1.523,0.014,0.011,0.0,0.0,0.0,0.0
4,4,0.012,-0.016,0.032,-1.525,0.012,0.011,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
301526,301526,0.012,-0.008,0.046,-1.486,-0.027,1.616,0.0,0.0,1.0,0.0
301527,301527,0.018,-0.007,0.038,-1.488,-0.028,1.613,0.0,0.0,1.0,0.0
301528,301528,0.010,-0.001,0.029,-1.490,-0.031,1.610,0.0,0.0,1.0,0.0
301529,301529,-0.007,0.005,0.025,-1.491,-0.033,1.608,0.0,0.0,1.0,0.0


In [32]:
driveraw = rawdata.to_numpy()#choose six feature
drivedata = driveraw[:, 1:7]
drivedata

array([[-5.000e-03,  8.000e-03,  1.800e-02, -1.523e+00,  1.500e-02,
         1.200e-02],
       [ 1.600e-02, -2.000e-03,  1.800e-02, -1.522e+00,  1.200e-02,
         1.200e-02],
       [ 3.700e-02, -5.000e-03,  1.800e-02, -1.520e+00,  1.400e-02,
         1.100e-02],
       ...,
       [ 1.000e-02, -1.000e-03,  2.900e-02, -1.490e+00, -3.100e-02,
         1.610e+00],
       [-7.000e-03,  5.000e-03,  2.500e-02, -1.491e+00, -3.300e-02,
         1.608e+00],
       [-6.000e-03,  1.500e-02,  1.900e-02, -1.494e+00, -3.400e-02,
         1.605e+00]])

In [33]:
lanechange_label = driveraw[:, 7]#choose label
breaking_label = driveraw[:, 9]

In [34]:
lanchange_model = KNeighborsRegressor(n_neighbors=100, p=2, weights='distance')#trian model
lanchange_model.fit(drivedata, lanechange_label)

KNeighborsRegressor(n_neighbors=100, weights='distance')

In [26]:
a = lanchange_model.predict([[-0.007,0.005,0.025,-1.491,-0.033,1.608]])
a

array([0.])

In [23]:
joblib.dump(lanchange_model, 'C:/Users/gordon_zhuang/Desktop/車用實習/imu machine learning/KNN_predict/lanchange_model')

['C:/Users/gordon_zhuang/Desktop/車用實習/imu machine learning/KNN_predict/lanchange_model']

In [24]:
breaking_model =  KNeighborsRegressor(n_neighbors=100, p=2, weights='distance')
breaking_model.fit(drivedata, breaking_label)

KNeighborsRegressor(n_neighbors=100, weights='distance')

In [28]:
a = breaking_model.predict([[-0.007,0.005,0.025,-1.491,-0.033,1.608]])
a

array([1.])

In [30]:
joblib.dump(breaking_model, 'C:/Users/gordon_zhuang/Desktop/車用實習/imu machine learning/KNN_predict/breaking_model')

['C:/Users/gordon_zhuang/Desktop/車用實習/imu machine learning/KNN_predict/breaking_model']

In [60]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.ensemble import RandomForestClassifier

In [61]:
lanechange_train_data, lanechange_test_data, lanechange_train_label, lanechange_test_label = train_test_split(drivedata, lanechange_label, test_size=0.5)
breaking_train_data, breaking_test_data, breaking_train_label, breaking_test_label = train_test_split(drivedata, breaking_label, test_size=0.5)

/////////////random forest training/////////

In [72]:
lanchange_RandomForest_model = RandomForestClassifier(n_estimators=1000, 
                                                      criterion='gini', 
                                                      max_depth=None,  
                                                      min_samples_split=2, 
                                                      min_samples_leaf=1,  
                                                      min_weight_fraction_leaf=0.0,  
                                                      max_features='auto',  
                                                      max_leaf_nodes=None, 
                                                      bootstrap=True,  
                                                      oob_score=True, 
                                                      n_jobs=1, 
                                                      random_state=None, 
                                                      verbose=0,  
                                                      warm_start=False, 
                                                      class_weight=None)

In [73]:
lanchange_RandomForest_model.fit(lanechange_train_data, lanechange_train_label)
print("%.4f" % lanchange_RandomForest_model.oob_score_)

0.9927


In [74]:
lanchange_RF_predict = lanchange_RandomForest_model.predict(lanechange_test_data)

In [75]:
report = classification_report(lanechange_test_label, lanchange_RF_predict, digits=4)#precision and recall
print(report)

              precision    recall  f1-score   support

        -2.0     1.0000    0.4922    0.6597       321
        -1.0     0.9927    0.6405    0.7786       854
         0.0     0.9922    1.0000    0.9961    148061
         1.0     0.9967    0.5468    0.7062      1110
         2.0     0.9956    0.5429    0.7026       420

    accuracy                         0.9922    150766
   macro avg     0.9955    0.6445    0.7687    150766
weighted avg     0.9923    0.9922    0.9912    150766



In [76]:
breaking_RandomForest_model = RandomForestClassifier(n_estimators=1000, 
                                                      criterion='gini', 
                                                      max_depth=None,  
                                                      min_samples_split=2, 
                                                      min_samples_leaf=1,  
                                                      min_weight_fraction_leaf=0.0,  
                                                      max_features='auto',  
                                                      max_leaf_nodes=None, 
                                                      bootstrap=True,  
                                                      oob_score=True, 
                                                      n_jobs=1, 
                                                      random_state=None, 
                                                      verbose=0,  
                                                      warm_start=False, 
                                                      class_weight=None)

In [77]:
breaking_RandomForest_model.fit(breaking_train_data,  breaking_train_label)
print("%.4f" % breaking_RandomForest_model.oob_score_)

0.9857


In [78]:
breaking_RF_predict = breaking_RandomForest_model.predict(breaking_test_data)

In [79]:
report = classification_report(breaking_test_label,breaking_RF_predict, digits=4)#precision and recall
print(report)

              precision    recall  f1-score   support

         0.0     0.9869    0.9992    0.9930    144588
         1.0     0.9486    0.6395    0.7640      4674
         2.0     0.9965    0.8086    0.8928      1426
         3.0     1.0000    0.8590    0.9241        78

    accuracy                         0.9862    150766
   macro avg     0.9830    0.8266    0.8935    150766
weighted avg     0.9858    0.9862    0.9849    150766



In [80]:
joblib.dump(lanchange_RandomForest_model, 'C:/Users/gordon_zhuang/Desktop/車用實習/imu machine learning/KNN_predict/lanchange_RandomForest_model')

['C:/Users/gordon_zhuang/Desktop/車用實習/imu machine learning/KNN_predict/lanchange_RandomForest_model']

In [81]:
joblib.dump(breaking_RandomForest_model, 'C:/Users/gordon_zhuang/Desktop/車用實習/imu machine learning/KNN_predict/breaking_RandomForest_model')

['C:/Users/gordon_zhuang/Desktop/車用實習/imu machine learning/KNN_predict/breaking_RandomForest_model']

/////////////XGBoost training/////////

In [51]:
lanechange_XGB_bodel = XGBClassifier(
                    n_estimators=1000,
                    learning_rate= 0.1, # 如同學習率
                    max_depth=6,                    
                    subsample=1, # 隨機取樣訓練樣本 訓練例項的子取樣比                    
                    gamma=0.1, # 用於控制是否後剪枝的引數,越大越保守，一般0.1、0.2這樣子                  
                    reg_lambda=1,  # 控制模型複雜度的權重值的L2正則化項引數，引數越大，模型越不容易過擬合。                    
                    max_delta_step=0,#最大增量步長，我們允許每個樹的權重估計。                 
                    colsample_bytree=1, # 生成樹時進行的列取樣 
                    # 這個引數預設是 1，是每個葉子裡面 h 的和至少是多少，對正負樣本不均衡時的 0-1 分類而言
                    # 假設 h 在 0.01 附近，min_child_weight 為 1 意味著葉子節點中最少需要包含 100 個樣本。
                    #這個引數非常影響結果，控制葉子節點中二階導的和的最小值，該引數值越小，越容易 overfitting。
                    min_child_weight=0.1,           
                    seed=1000  #隨機種子
                    #reg_alpha=0,  # L1 正則項引數                    
                    #scale_pos_weight=1, #如果取值大於0的話，在類別樣本不平衡的情況下有助於快速收斂。平衡正負權重                    
                    #objective= 'multi:softmax', #多分類的問題 指定學習任務和相應的學習目標
                    #num_class=10,  # 類別數，多分類與 multisoftmax 並用
                    #silent=0 ,  # 設定成1則沒有執行資訊輸出，最好是設定為0.是否在執行升級時列印訊息。  
                    #nthread=4, # cpu 執行緒數 預設最大
                    #eval_metric= 'auc'
)

In [52]:
lanechange_XGB_bodel.fit(lanechange_train_data, lanechange_train_label)

[10:49:48] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0.1, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=6,
              min_child_weight=0.1, missing=nan, monotone_constraints='()',
              n_estimators=1000, n_jobs=12, num_parallel_tree=1,
              objective='multi:softprob', random_state=1000, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, seed=1000, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [53]:
lanchange_XGB_predict = lanechange_XGB_bodel.predict(lanechange_test_data)

In [54]:
report = classification_report(lanechange_test_label,lanchange_XGB_predict, digits=4)#precision and recall
print(report)

              precision    recall  f1-score   support

        -2.0     0.9489    0.6799    0.7922       328
        -1.0     0.9882    0.6834    0.8080       859
         0.0     0.9941    0.9998    0.9969    148020
         1.0     0.9903    0.6240    0.7656      1149
         2.0     0.9629    0.8220    0.8868       410

    accuracy                         0.9939    150766
   macro avg     0.9769    0.7618    0.8499    150766
weighted avg     0.9938    0.9939    0.9933    150766



In [55]:
breaking_XGB_bodel = XGBClassifier(
                    n_estimators=1000,
                    learning_rate= 0.1, # 如同學習率
                    max_depth=3,                    
                    subsample=1, # 隨機取樣訓練樣本 訓練例項的子取樣比                    
                    gamma=0.1, # 用於控制是否後剪枝的引數,越大越保守，一般0.1、0.2這樣子                  
                    reg_lambda=1,  # 控制模型複雜度的權重值的L2正則化項引數，引數越大，模型越不容易過擬合。                    
                    max_delta_step=0,#最大增量步長，我們允許每個樹的權重估計。                 
                    colsample_bytree=1, # 生成樹時進行的列取樣 
                    # 這個引數預設是 1，是每個葉子裡面 h 的和至少是多少，對正負樣本不均衡時的 0-1 分類而言
                    # 假設 h 在 0.01 附近，min_child_weight 為 1 意味著葉子節點中最少需要包含 100 個樣本。
                    #這個引數非常影響結果，控制葉子節點中二階導的和的最小值，該引數值越小，越容易 overfitting。
                    min_child_weight=0.1,           
                    seed=1000  #隨機種子
                    #reg_alpha=0,  # L1 正則項引數                    
                    #scale_pos_weight=1, #如果取值大於0的話，在類別樣本不平衡的情況下有助於快速收斂。平衡正負權重                    
                    #objective= 'multi:softmax', #多分類的問題 指定學習任務和相應的學習目標
                    #num_class=10,  # 類別數，多分類與 multisoftmax 並用
                    #silent=0 ,  # 設定成1則沒有執行資訊輸出，最好是設定為0.是否在執行升級時列印訊息。  
                    #nthread=4, # cpu 執行緒數 預設最大
                    #eval_metric= 'auc'
)

In [56]:
breaking_XGB_bodel.fit(drivedata, breaking_label)

[10:51:45] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0.1, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=0.1, missing=nan, monotone_constraints='()',
              n_estimators=1000, n_jobs=12, num_parallel_tree=1,
              objective='multi:softprob', random_state=1000, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, seed=1000, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [57]:
breaking_XGB_predict = breaking_XGB_bodel.predict(breaking_test_data)

In [58]:
report = classification_report(breaking_test_label,breaking_XGB_predict, digits=4)#precision and recall
print(report)

              precision    recall  f1-score   support

         0.0     0.9814    0.9985    0.9899    144555
         1.0     0.9124    0.4770    0.6265      4717
         2.0     0.9800    0.7972    0.8792      1415
         3.0     1.0000    1.0000    1.0000        79

    accuracy                         0.9803    150766
   macro avg     0.9685    0.8182    0.8739    150766
weighted avg     0.9792    0.9803    0.9775    150766



In [44]:
joblib.dump(lanechange_XGB_bodel, 'C:/Users/gordon_zhuang/Desktop/車用實習/imu machine learning/KNN_predict/lanechange_XGB_model')

['C:/Users/gordon_zhuang/Desktop/車用實習/imu machine learning/KNN_predict/lanechange_XGB_model']

In [45]:
joblib.dump(breaking_XGB_bodel, 'C:/Users/gordon_zhuang/Desktop/車用實習/imu machine learning/KNN_predict/breaking_XGB_model')

['C:/Users/gordon_zhuang/Desktop/車用實習/imu machine learning/KNN_predict/breaking_XGB_model']

In [46]:
type(breaking_test_data)

numpy.ndarray